# Predicting the dating score from three-word description

## Data processing

In [37]:
from fastai.imports import *

In [38]:
df = pd.read_excel('processed_data hand-corrected.xlsx')

In [39]:
df.head()

,url,hyperlink,date,A_name,A_gender,B_name,B_gender,A_what_were_you_hoping_for,A_first_impressions,A_what_did_you_talk_about,...,B_marks_out_of_10,B_would_you_meet_again,B_meet_again_bool,A_any_awkward_moments,B_best_thing_about_X,B_any_awkward_moments,A_and_did_you_kiss,B_and_did_you_kiss,A_marks_out_of_10_float,B_marks_out_of_10_float
0,https://www.theguardian.com/lifeandstyle/2022/dec/03/blind-date-olivia-shawn,https://www.theguardian.com/lifeandstyle/2022/dec/03/blind-date-olivia-shawn,2022-12-03,olivia,f,shawn,m,"A nice evening with good conversation, or at least a good story.","Tall, warm and easy to talk to. He was late, but in his defence, I had said I was going to be late and then I wasn’t.",Sorting recycling at Glastonbury. The moral dilemmas of our respective jobs. Dogs. Reasons to avoid the sea. The value of good storytelling.,...,A solid 8. She’s a sweetheart.,It would be fun to hang out next time I’m in the Big Smoke.,y,NaN,NaN,NaN,NaN,NaN,7.9,8.0
1,https://www.theguardian.com/lifeandstyle/2022/nov/26/blind-date-calum-ciaran,https://www.theguardian.com/lifeandstyle/2022/nov/26/blind-date-calum-ciaran,2022-11-26,calum,m,ciaran,m,"A rich, handsome future husband whose salary I could retire on at 30 and devote myself to fun stuff. More realistically, a fun evening with some nice company.","Ciarán was lovely! I liked his smile. We hugged spontaneously, like old pals.",Our trips to Colombia. Using Scottish/Irish slang in England. Our love of This Is My House. Our mutual distrust of anyone who eats green bananas.,...,"9, couldn’t fault him.","I didn’t feel a romantic spark, but we agreed over text afterwards to be pals.",f,NaN,NaN,NaN,NaN,NaN,8.0,9.0
2,https://www.theguardian.com/lifeandstyle/2022/nov/19/blind-date-andrew-kath,https://www.theguardian.com/lifeandstyle/2022/nov/19/blind-date-andrew-kath,2022-11-19,andrew,f,kath,f,"To meet someone new. I am open to retiring anywhere so met Kath in Birmingham, halfway between her home and mine. I’m willing to travel to meet someone special.",Enthusiasm. We talked easily and happily.,"Our children. Food. Animal welfare. Green issues. Brexit. Electric cars. Natural disasters. Faith. The Proms. The sudden death of her husband, and my divorce. Empty nest syndrome.",...,8,We swapped numbers.,y,I may have asked too many questions but Andrew was always very polite and answered them.,"He is a very sensitive gentleman who cares deeply about his family, a value I respect and share. He also takes pleasure in the simple things in life, as do I.",NaN,NaN,NaN,8.0,8.0
3,https://www.theguardian.com/lifeandstyle/2022/nov/12/blind-date-joseph-cole,https://www.theguardian.com/lifeandstyle/2022/nov/12/blind-date-joseph-cole,2022-11-12,joseph,m,cole,m,To meet someone new and have a nice meal.,He was cute – great hair – and better than I was expecting. I got lost so was a bit late and he was already there.,Differences between the UK and the US. Politics. Pubs. Favourite movies (Toy Story 2). University … He was really easy to talk to.,...,A well-earned 7.,I would.,y,NaN,He’s unafraid to talk about what he’s passionate about.,NaN,NaN,NaN,8.0,7.0
4,https://www.theguardian.com/lifeandstyle/2022/nov/05/blind-date-dick-nicole,https://www.theguardian.com/lifeandstyle/2022/nov/05/blind-date-dick-nicole,2022-11-05,dick,m,nicole,f,To meet a new friend and have a great meal.,"Open, vivacious, lively and very friendly.","Our families. Our mutual interest in sketching, drawing and painting – we even showed each other our sketchbooks. Nicole’s were full of lovely studies of people.",...,NaN,Unlikely. There was no flirting. A compliment or two would’ve been nice! He was charmingly flirtatious with the ladies on the table behind at one point – he needs to bring that twinkle to his date. I also think he would prefer that I live locally.,n,NaN,"Unsurprisingly for an architect he draws really well, and I suspect his watercolours are charming. He draws really well – I was delighted to find we both carry a sketchb

In [40]:
df_a = df.rename(columns={'A_describe_X_in_three_words': 'text', 'A_marks_out_of_10_float': 'score'})[['text', 'score']]
df_b = df.rename(columns={'B_describe_X_in_three_words': 'text', 'B_marks_out_of_10_float': 'score'})[['text', 'score']]
new_df = pd.concat([df_a, df_b], axis=0)
new_df

,text,score
0,"Good-humoured, creative, curious.",7.9
1,"Bubbly, fun, intelligent.",8.0
2,"Enthusiastic, energetic, musical.",8.0
3,"Smart, American, sweet.",8.0
4,"Lively, open and vivacious.",NaN
...,...,...
714,NaN,7.0
715,NaN,7.0
716,NaN,9.0
717,NaN,8.0


In [41]:
new_df = new_df.dropna().reset_index(drop=True)
new_df

,text,score
0,"Good-humoured, creative, curious.",7.9
1,"Bubbly, fun, intelligent.",8.0
2,"Enthusiastic, energetic, musical.",8.0
3,"Smart, American, sweet.",8.0
4,"We discussed this during the date: energetic (not chaotic), fun, honest.",8.0
...,...,...
817,"Bright, bubbly and bonkers (in a good way).",7.0
818,"Erudite, stunning, adventurous.",10.0
819,"Kind, funny, intelligent.",7.0
820,"Polite, engaging, carnivore.",7.0


In [42]:
new_df.isna().sum()

text     0
score    0
dtype: int64

In [43]:
new_df.loc[:, 'bool'] = new_df.loc[:, 'score'] >= 8
new_df

,text,score,bool
0,"Good-humoured, creative, curious.",7.9,False
1,"Bubbly, fun, intelligent.",8.0,True
2,"Enthusiastic, energetic, musical.",8.0,True
3,"Smart, American, sweet.",8.0,True
4,"We discussed this during the date: energetic (not chaotic), fun, honest.",8.0,True
...,...,...,...
817,"Bright, bubbly and bonkers (in a good way).",7.0,False
818,"Erudite, stunning, adventurous.",10.0,True
819,"Kind, funny, intelligent.",7.0,False
820,"Polite, engaging, carnivore.",7.0,False


## from-scratch model

In [9]:
from torch import tensor

y = tensor(new_df['bool'].values).float()
y.shape

torch.Size([822])

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english')
X = tensor(vectorizer.fit_transform(new_df['text']).toarray()).float()
X.shape

torch.Size([822, 597])

In [11]:
print(vectorizer.get_feature_names_out()[X[0].numpy().nonzero()])
new_df.iloc[0]

['creative' 'curious' 'good' 'humoured']


text     Good-humoured, creative, curious.
score                                  7.9
bool                                 False
Name: 0, dtype: object

In [12]:
import torch
# torch.manual_seed(0)
n_coeffs = X.shape[1]
coeffs = torch.rand(n_coeffs, 1) - 0.5
coeffs

tensor([[ 2.3134e-01],
        [-3.4916e-01],
        [-6.3377e-02],
        [-3.2861e-01],
        [-3.7507e-01],
        [-3.7051e-02],
        [ 5.3190e-02],
        [-2.2156e-01],
        [ 4.8554e-01],
        [ 2.1481e-01],
        [ 1.4890e-01],
        [-2.6573e-03],
        [ 1.2481e-01],
        [-1.6249e-01],
        [ 2.9113e-01],
        [ 4.2867e-01],
        [ 1.1080e-01],
        [ 4.2263e-01],
        [ 4.5667e-01],
        [ 2.9341e-01],
        [-2.1037e-01],
        [ 1.2086e-02],
        [ 2.9181e-01],
        [ 3.9918e-01],
        [ 1.1695e-01],
        [ 4.1202e-02],
        [-1.6139e-01],
        [ 2.9334e-01],
        [-9.7253e-02],
        [ 1.6498e-01],
        [ 3.2328e-01],
        [-3.5465e-01],
        [-1.3714e-01],
        [ 3.9398e-01],
        [ 1.7668e-01],
        [ 1.6297e-01],
        [ 2.4948e-01],
        [ 3.2014e-01],
        [-2.3448e-01],
        [-3.7022e-01],
        [-1.0971e-01],
        [-1.8980e-01],
        [-2.5936e-01],
        [-4

In [13]:
def calc_pred(X, coeffs):
    return torch.sigmoid(X @ coeffs)

def calc_loss(pred, y):
    return torch.abs(pred - y).mean()

def init_coeffs():
    return (torch.rand(n_coeffs, 1)*0.1).requires_grad_()

def update_coeffs(coeffs, lr):
    coeffs.sub_(lr * coeffs.grad)
    coeffs.grad.zero_()

def accuracy(coeffs, X, y):
    preds = calc_pred(X, coeffs)
    return (y.bool() == (preds > 0.5)).float().mean()

def one_epoch(coeffs, lr, X, y):
    loss = calc_loss(calc_pred(X, coeffs), y)
    loss.backward()
    with torch.no_grad(): update_coeffs(coeffs, lr)
    print(f"Loss: {loss:.4f}, Accuracy {accuracy(coeffs, X, y):.4f}", end=' ')

In [14]:
from fastai.data.transforms import RandomSplitter
trn_split, val_split = RandomSplitter()(new_df) # can set seed
X_train, X_val = X[trn_split], X[val_split]
y_train, y_val = y[trn_split][:, None], y[val_split][:, None]
len(y_train), len(y_val)

(658, 164)

In [15]:
def train_model(epochs=30, lr=0.1, X=X_train, y=y_train):
    # torch.manual_seed(0)
    coeffs = init_coeffs()
    for i in range(epochs):
        print(f"Epoch {i+1}: ", end='')
        one_epoch(coeffs, lr, X, y)
        print()
    return coeffs

In [16]:
coeffs = train_model(lr=4)

Epoch 1: Loss: 0.4982, Accuracy 0.5350 
Epoch 2: Loss: 0.4955, Accuracy 0.5365 
Epoch 3: Loss: 0.4929, Accuracy 0.5410 
Epoch 4: Loss: 0.4903, Accuracy 0.5456 
Epoch 5: Loss: 0.4877, Accuracy 0.5547 
Epoch 6: Loss: 0.4852, Accuracy 0.5714 
Epoch 7: Loss: 0.4827, Accuracy 0.5866 
Epoch 8: Loss: 0.4802, Accuracy 0.5988 
Epoch 9: Loss: 0.4777, Accuracy 0.6064 
Epoch 10: Loss: 0.4753, Accuracy 0.6094 
Epoch 11: Loss: 0.4729, Accuracy 0.6170 
Epoch 12: Loss: 0.4706, Accuracy 0.6216 
Epoch 13: Loss: 0.4683, Accuracy 0.6216 
Epoch 14: Loss: 0.4661, Accuracy 0.6216 
Epoch 15: Loss: 0.4639, Accuracy 0.6246 
Epoch 16: Loss: 0.4617, Accuracy 0.6292 
Epoch 17: Loss: 0.4596, Accuracy 0.6337 
Epoch 18: Loss: 0.4576, Accuracy 0.6353 
Epoch 19: Loss: 0.4556, Accuracy 0.6429 
Epoch 20: Loss: 0.4536, Accuracy 0.6444 
Epoch 21: Loss: 0.4517, Accuracy 0.6444 
Epoch 22: Loss: 0.4498, Accuracy 0.6489 
Epoch 23: Loss: 0.4480, Accuracy 0.6520 
Epoch 24: Loss: 0.4462, Accuracy 0.6550 
Epoch 25: Loss: 0.4444, A

In [17]:
def show_coeffs():
    return dict(zip(vectorizer.get_feature_names_out(), coeffs.requires_grad_(False).numpy().flatten()))

In [18]:
weights_df = pd.DataFrame.from_dict(show_coeffs(), orient='index', columns=['weight'])
weights_df.sort_values('weight', ascending=False, inplace=True)
weights_df.reset_index(inplace=True)
weights_df.rename(columns={'index': 'word'}, inplace=True)
weights_df.to_excel('weights.xlsx', index=False)
weights_df

,word,weight
0,fun,0.946524
1,funny,0.867420
2,engaging,0.803172
3,interesting,0.553711
4,warm,0.450060
...,...,...
592,going,-0.335502
593,easy,-0.349811
594,talkative,-0.491349
595,friendly,-0.605620


In [19]:
val_preds = calc_pred(X_val, coeffs)
results = y_val.bool() == (val_preds > 0.5)
results.float().mean()

tensor(0.6524)

## Using transformers

In [35]:
from datasets import *
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [44]:
ds = Dataset.from_pandas(new_df)
ds = ds.rename_column('bool', 'labels')
ds = ds.cast_column('labels', Value('float32'))
dds = ds.train_test_split(seed=2005)

Casting the dataset:   0%|          | 0/822 [00:00<?, ? examples/s]

In [45]:
model_name = 'microsoft/deberta-v3-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
d:\Users\selva\Documents\Coding\GuardianBlindDate\venv\lib\site-packages\transformers\convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [46]:
def tokenize_func(examples):
    return tokenizer(examples['text'], truncation=True, padding=True)
dds = dds.map(tokenize_func, batched=True)

Map:   0%|          | 0/616 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/206 [00:00<?, ? examples/s]

In [47]:
import evaluate
metric = evaluate.load('accuracy')
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.round(logits)
    return metric.compute(predictions=preds, references=labels)


In [48]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.bias', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [58]:
bs = 16
lr = 1e-5
epochs = 16
# args = TrainingArguments('outputs', learning_rate=lr, per_device_train_batch_size=bs, per_device_eval_batch_size=bs, num_train_epochs=epochs,
#                          evaluation_strategy='epoch', save_strategy='epoch', weight_decay=0.01, warmup_ratio=0.1, load_best_model_at_end=True, metric_for_best_model='accuracy'
#                         ) # auto_find_batch_size? (requires pip install accelerate)

args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=False,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

PyTorch: setting up devices


In [59]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dds['train'],
    eval_dataset=dds['test'],
    compute_metrics=compute_metrics,
)

In [60]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: score, text. If score, text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
d:\Users\selva\Documents\Coding\GuardianBlindDate\venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 616
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 156
  Number of trainable parameters = 141895681


  0%|          | 0/156 [00:00<?, ?it/s]

The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: score, text. If score, text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 206
  Batch size = 32


  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.27644243836402893, 'eval_accuracy': 0.6456310679611651, 'eval_runtime': 4.06, 'eval_samples_per_second': 50.739, 'eval_steps_per_second': 1.724, 'epoch': 1.0}


...

The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: score, text. If score, text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 206
  Batch size = 32


  0%|          | 0/7 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 0.3007143437862396, 'eval_accuracy': 0.6407766990291263, 'eval_runtime': 4.166, 'eval_samples_per_second': 49.448, 'eval_steps_per_second': 1.68, 'epoch': 4.0}
{'train_runtime': 315.2646, 'train_samples_per_second': 7.816, 'train_steps_per_second': 0.495, 'train_loss': 0.07165377567975949, 'epoch': 4.0}


TrainOutput(global_step=156, training_loss=0.07165377567975949, metrics={'train_runtime': 315.2646, 'train_samples_per_second': 7.816, 'train_steps_per_second': 0.495, 'train_loss': 0.07165377567975949, 'epoch': 4.0})

In [51]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-small')

loading configuration file config.json from cache at C:\Users\selva/.cache\huggingface\hub\models--microsoft--deberta-v3-small\snapshots\a36c739020e01763fe789b4b85e2df55d6180012\config.json
Model config DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-small",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "transformers_version": "4.26.1",
  "type_vocab_size": 0,
  "vocab_size": 12810

In [65]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer) # you can infer the task if the model hasn't been fine-tuned, I think

text = "interesting"
classifier(text)

[{'label': 'LABEL_0', 'score': 0.7054358124732971}]

In [66]:
for example in dds['test']:
    print(example['text'], classifier(text), example['labels'])

Engaging, interesting, cool. [{'label': 'LABEL_0', 'score': 0.7054358124732971}] 1.0
Charming, hard-working, good guy. [{'label': 'LABEL_0', 'score': 0.7054358124732971}] 1.0
Witty, smart, hard-working. [{'label': 'LABEL_0', 'score': 0.7054358124732971}] 1.0
Funny, chatty, polite. [{'label': 'LABEL_0', 'score': 0.7054358124732971}] 0.0
 Bubbly, pretty, outdoorsy. [{'label': 'LABEL_0', 'score': 0.7054358124732971}] 1.0
Attentive, kind, interesting. [{'label': 'LABEL_0', 'score': 0.7054358124732971}] 1.0
Funky roller chick. [{'label': 'LABEL_0', 'score': 0.7054358124732971}] 0.0
Up for it (a fun and free night). [{'label': 'LABEL_0', 'score': 0.7054358124732971}] 1.0
Delightful, insomniac, creative. [{'label': 'LABEL_0', 'score': 0.7054358124732971}] 1.0
Sensitive, adventurous, on the brink of something new? [{'label': 'LABEL_0', 'score': 0.7054358124732971}] 1.0
Happy, interesting, energetic. [{'label': 'LABEL_0', 'score': 0.7054358124732971}] 1.0
Three probably isn’t fair. Grounded, cu

0                                             Good-humoured, creative, curious.
1                                                     Bubbly, fun, intelligent.
2                                             Enthusiastic, energetic, musical.
3                                                       Smart, American, sweet.
4      We discussed this during the date: energetic (not chaotic), fun, honest.
                                         ...                                   
817                                 Bright, bubbly and bonkers (in a good way).
818                                             Erudite, stunning, adventurous.
819                                                   Kind, funny, intelligent.
820                                                Polite, engaging, carnivore.
821                                                         Lively, happy, fun.
Name: text, Length: 822, dtype: object

In [27]:
tokenizer("funny, interesting, fun", return_tensors="pt")

{'input_ids': tensor([[   1, 3270,  261, 1257,  261,  785,    2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}